# Tests of coordinate system effects on shear profiles

Author:  Tomomi Sunayama, July 15, 2024

Tested and documented by: Camille Avestruz, July 16, 2024


This notebook illustrates the impact of having an incorrect coordinate system when calculating shear profiles.  The default coordinate system for `CLMM` is the pixel coordinate system.  This is consistent with DC2 catalogs.  However, if we input a catalog that assumes a sky coordinate system and use the default pixel coordinate system (or vice versa), the signal of shear around a cluster disappears because the signal essentially looks random.


We test:
- CosmoDC2 source galaxies with shears extracted from `TXPipe` for a single galaxy cluster (pixel coordinate system)
- Example source galaxies for galaxy clusters from a [Summer School](https://github.com/oguri/wlcluster_tutorial) taught by Masamune Oguri (pixel coordinate system)
- HSC Y3 source galaxies with shears post processed by Tomomi Sunayama (sky coordinate system)

Before running this notebook, you will need to download some data.  The data is available through a [dropbox link](https://www.dropbox.com/scl/fo/dwsccslr5iwb7lnkf8jvx/AJkjgFeemUEHpHaZaHHqpAg?rlkey=efbtsr15mdrs3y6xsm7l48o0r&st=xb58ap0g&dl=0)

First, create a directory where you want to put the example data, e.g.
```
data_coords_dir="~/CLMM_test_data/data_coords"
mkdir -p $data_coords_dir
cd $data_coords_dir
```
Download all files from [dropbox link](https://www.dropbox.com/scl/fo/dwsccslr5iwb7lnkf8jvx/AJkjgFeemUEHpHaZaHHqpAg?rlkey=efbtsr15mdrs3y6xsm7l48o0r&st=xb58ap0g&dl=0).  This will be a zip file, `CLMM_data.zip` of size 242 Mb. scp or move this to `$data_coords_dir`.

From the directory, you should be able to unzip:
```
unzip data_CLMM.zip -d .
```
You now have the necessary data files to run this notebook.  Make sure to change the `data_coords_dir` variable in the cell below to the appropriate location where you unzipped these files.


In [3]:
#  CHANGE THIS TO YOUR LOCATION
data_coords_dir="/pscratch/sd/a/avestruz/clmm-area/CLMM_test_data/data_coords/"

In [4]:
import numpy as np
import matplotlib.pyplot as plt

# %matplotlib inline
from astropy.table import Table
import pickle as pkl
from pathlib import Path
import pandas
from astropy.io import fits

In [5]:
from clmm import Cosmology

cosmo = Cosmology(H0=70.0, Omega_dm0=0.27 - 0.045, Omega_b0=0.045, Omega_k0=0.0)

## Example galaxy cluster from CosmoDC2

Here, we plot an example galaxy cluster shear profile using `clmm`.  The cluster and source galaxy files are generated from the CosmoDC2 processed through TXPipe.  We test the coordinate system.

In [7]:
cluster = pandas.read_pickle(data_coords_dir+'test_cluster.pkl')

In [8]:
cluster_z = cluster['redshift']  # Cluster redshift
cluster_ra = cluster['ra']  # Cluster Ra in deg
cluster_dec = cluster['dec']

In [10]:
source = np.genfromtxt(data_coords_dir+'test_source.txt',names=True)

In [15]:
gal_ra = source['ra']
gal_dec = source['dec']
gal_e1 = source['e1']
gal_e2 = source['e2']
gal_z = source['zmean']
gal_id = np.arange(len(source['ra']))

In [16]:
import clmm
import clmm.dataops as da
from clmm.utils import convert_units

# Create a GCData with the galaxies.
galaxies = clmm.GCData(
    [gal_ra, gal_dec, gal_e1, gal_e2, gal_z, gal_id], names=["ra", "dec", "e1", "e2", "z", "id"]
)

Here we create a `GalaxyCluster` object, specifying an *incorrect* coordinate system.  For source galaxies from CosmoDC2, these are in the **pixel** coordinate system.  We use the implemented kwarg when defining the galaxy cluster object to specify the **sky** coordinate system.

In [17]:
# Create a GalaxyCluster.
cluster = clmm.GalaxyCluster("Name of cluster", cluster_ra, cluster_dec, cluster_z, galaxies)

# Convert elipticities into shears for the members.
cluster.compute_tangential_and_cross_components(coordinate_system='sky')
print(cluster.galcat.colnames)

# Measure profile and add profile table to the cluster.
seps = convert_units(cluster.galcat["theta"], "radians", "Mpc", cluster.z, cosmo)

cluster.make_radial_profile(
    bins=da.make_bins(0.1, 3.0, 15, method="evenlog10width"),
    bin_units="Mpc",
    cosmo=cosmo,
    include_empty_bins=False,
    gal_ids_in_bins=True,
)
print(cluster.profile.colnames)

TypeError: GalaxyCluster.compute_tangential_and_cross_components() got an unexpected keyword argument 'coordinate_system'

Here, we plot the resulting profile when `clmm` uses assumes a coordinate system inconsistent with the catalogs provided.  You will note that the signal is virtually zero at almost all radii.

In [ ]:
fig = plt.figure(figsize=(6, 4))
ax = fig.add_axes((0, 0, 1, 1))
errorbar_kwargs = dict(linestyle="", marker="o", markersize=1, elinewidth=0.5, capthick=0.5)
ax.errorbar(
    cluster.profile["radius"],
    cluster.profile["gt"],
    cluster.profile["gt_err"],
    c="k",
    **errorbar_kwargs
)
ax.set_xlabel("r [Mpc]", fontsize=10)
ax.set_ylabel(r"$g_t$", fontsize=10)
ax.grid(lw=0.3)
ax.minorticks_on()
ax.grid(which="minor", lw=0.1)
plt.show()

Here we create a `GalaxyCluster` object, specifying the *correct* coordinate system.  For source galaxies from CosmoDC2, these are in the **pixel** coordinate system.  We use the implemented kwarg when defining the galaxy cluster object to also specify the **pixel** coordinate system.

In [ ]:
cluster2 = clmm.GalaxyCluster("Name of cluster", cluster_ra, cluster_dec, cluster_z, galaxies)
cluster2.compute_tangential_and_cross_components(coordinate_system='pixel')
print(cluster.galcat.colnames)

# Measure profile and add profile table to the cluster.
seps = convert_units(cluster2.galcat["theta"], "radians", "Mpc", cluster2.z, cosmo)

cluster2.make_radial_profile(
    bins=da.make_bins(0.1, 3.0, 15, method="evenlog10width"),
    bin_units="Mpc",
    cosmo=cosmo,
    include_empty_bins=False,
    gal_ids_in_bins=True,
)
print(cluster.profile.colnames)
fig = plt.figure(figsize=(6, 4))
ax = fig.add_axes((0, 0, 1, 1))
errorbar_kwargs = dict(linestyle="", marker="o", markersize=1, elinewidth=0.5, capthick=0.5)
ax.errorbar(
    cluster.profile["radius"],
    cluster.profile["gt"],
    cluster.profile["gt_err"],
    label='sky'   
)
ax.errorbar(
    cluster2.profile["radius"],
    cluster2.profile["gt"],
    cluster2.profile["gt_err"],
    label='pixel'
)
ax.set_xlabel("r [Mpc]", fontsize=20)
ax.set_ylabel(r"$g_t$", fontsize=20)
ax.grid(lw=0.3)
ax.minorticks_on()
ax.grid(which="minor", lw=0.1)
plt.legend(fontsize=20)
plt.show()

## Example source galaxies from M. Oguri

This dataset is a curated selection of cluster and source catalogs from Summer School lectures delivered by Masamune Oguri.  There are eight galaxy clusters in this selection.  

More details on the corresponding tutorial can be found at this [GitHub link](https://github.com/oguri/wlcluster_tutorial). The coordinate system is pixel.  These are also in the **pixel** coordinate system.

In [18]:
clusters = ['a1703', 'gho1320', 'sdss0851', 'sdss1050', 
            'sdss1138', 'sdss1226', 'sdss1329', 'sdss1531']

zl_all = {'a1703': 0.277, 'gho1320': 0.308, 'sdss0851': 0.370, 'sdss1050': 0.60, 
          'sdss1138': 0.451, 'sdss1226': 0.435, 'sdss1329': 0.443, 'sdss1531': 0.335}

ra_cl_all = {'a1703': 198.771833, 'gho1320': 200.703208, 'sdss0851': 132.911917, 'sdss1050': 162.666250, 
             'sdss1138': 174.537292, 'sdss1226': 186.712958, 'sdss1329': 202.393708, 'sdss1531':  232.794167}

dec_cl_all = {'a1703': 51.817389, 'gho1320': 31.654944, 'sdss0851': 33.518361, 'sdss1050': 0.285306, 
              'sdss1138': 27.908528, 'sdss1226': 21.831194, 'sdss1329': 22.721167, 'sdss1531':  34.240278}

In [19]:
cname   = 'a1703'

# cluster redshift
zl      = zl_all.get(cname)

# coordinates of the cluster center
ra_cl   = ra_cl_all.get(cname)
dec_cl  = dec_cl_all.get(cname)

# fix source redshift to 1.0
zs = 1.0

We inspect the first galaxy cluster, Abell 1703.

In [21]:
rfile = data_coords_dir+'data/shear_' + cname + '.dat'
data = np.loadtxt(rfile, comments = '#')

ra  = data[:, 0]
dec = data[:, 1]
e1  = data[:, 2]
e2  = data[:, 3]
wei = data[:, 4]
ids = np.arange(np.shape(data)[0])
redshifts=np.ones(np.shape(data)[0])
galaxies = clmm.GCData(
    [ra, dec, e1, e2, redshifts, ids], names=["ra", "dec", "e1", "e2", "z", "id"]
)

Here we create a GalaxyCluster object, specifying the correct coordinate system. For source galaxies from the Oguri curated dataset, these are in the pixel coordinate system. We use the implemented kwarg when defining the galaxy cluster object to also specify the pixel coordinate system.

In [22]:
cluster = clmm.GalaxyCluster("Name of cluster", ra_cl, dec_cl, zl, galaxies)

# Convert elipticities into shears for the members.
cluster.compute_tangential_and_cross_components(coordinate_system='pixel')
print(cluster.galcat.colnames)

# Measure profile and add profile table to the cluster.
seps = convert_units(cluster.galcat["theta"], "radians", "Mpc", cluster.z, cosmo)

cluster.make_radial_profile(
    bins=da.make_bins(0.2, 3.0, 8, method="evenlog10width"),
    bin_units="Mpc",
    cosmo=cosmo,
    include_empty_bins=False,
    gal_ids_in_bins=True,
)
print(cluster.profile.colnames)


fig = plt.figure(figsize=(6, 4))
ax = fig.add_axes((0, 0, 1, 1))
errorbar_kwargs = dict(linestyle="", marker="o", markersize=1, elinewidth=0.5, capthick=0.5)
ax.errorbar(
    cluster.profile["radius"],
    cluster.profile["gt"],
    cluster.profile["gt_err"],
    label='pixel'
)


cluster.compute_tangential_and_cross_components(coordinate_system='sky')
print(cluster.galcat.colnames)

# Measure profile and add profile table to the cluster.
seps = convert_units(cluster.galcat["theta"], "radians", "Mpc", cluster.z, cosmo)

cluster.make_radial_profile(
    bins=da.make_bins(0.1, 3.0, 8, method="evenlog10width"),
    bin_units="Mpc",
    cosmo=cosmo,
    include_empty_bins=False,
    gal_ids_in_bins=True,
)
print(cluster.profile.colnames)

ax.errorbar(
    cluster.profile["radius"],
    cluster.profile["gt"],
    cluster.profile["gt_err"],
    label='sky'
)


ax.set_xlabel("r [Mpc]", fontsize=20)
ax.set_ylabel(r"$g_t$", fontsize=20)
ax.grid(lw=0.3)
ax.minorticks_on()
ax.grid(which="minor", lw=0.1)
plt.legend(fontsize=20)
plt.show()

TypeError: GalaxyCluster.compute_tangential_and_cross_components() got an unexpected keyword argument 'coordinate_system'

## Example source galaxies from HSC Y3

This dataset is a simplified version of HSC Y3 data (GAMA15H), post-processed by Tomomi Sunayama for testing purposes.  The pre-processed data is already public. These catalogs assume a **sky** pixel coordinate system.

In [24]:
clust = np.genfromtxt(data_coords_dir+'GAMA15H/redmapper_dr8_GAMA15H.txt',dtype=np.dtype([('ra',np.float64),('dec',np.float64),('z',np.float64),('richness',np.float64)]))

In [25]:
source = fits.getdata(data_coords_dir+'GAMA15H/GAMA15H_tutorial.fits')

In [26]:
cl = clust[0]

In [27]:
from scipy import spatial
source1 = source[source['photoz']>(cl['z']+0.3)]
tree = spatial.cKDTree(np.array((source1['ra'],source1['dec'])).T)
sel = tree.query_ball_point([cl['ra'],cl['dec']], 3)
bg = source1[sel]

In [ ]:
bg

In [ ]:
sources = clmm.GCData(
    [bg['RA'], bg['Dec'], bg['e1'], bg['e2'], bg['photoz'],bg['weight']], names=["ra", "dec", "e1", "e2", "z", "w_ls"]
)

In [ ]:
cluster = clmm.GalaxyCluster("redmapper", cl['ra'], cl['dec'], cl['z'], sources)

In [ ]:
sigma_c = cosmo.eval_sigma_crit(cl['z'], sources['z'])

In [ ]:
cluster.compute_tangential_and_cross_components(
        shape_component1="e1",
        shape_component2="e2",
        tan_component="DS_t",
        cross_component="DS_x",
        cosmo=cosmo,
        is_deltasigma=True,
        use_pdz=False,
        coordinate_system='sky',
    )



In [ ]:
cluster

In [ ]:
seps = convert_units(cluster.galcat["theta"], "radians", "Mpc", cluster.z, cosmo)

cluster.make_radial_profile(
    tan_component_in="DS_t",
    cross_component_in="DS_x",
    tan_component_out="DS_t",
    cross_component_out="DS_x",
    weights_in="w_ls",
    bins=da.make_bins(0.1, 20.0, 15, method="evenlog10width"),
    bin_units="Mpc",
    cosmo=cosmo,
    include_empty_bins=False,
    gal_ids_in_bins=False,
)
print(cluster.profile.colnames)



In [ ]:

fig = plt.figure(figsize=(6, 6))
ax = fig.add_axes((0, 0, 1, 1))
errorbar_kwargs = dict(linestyle="", marker="o", markersize=1, elinewidth=0.5, capthick=0.5)
ax.errorbar(
    cluster.profile["radius"],
    cluster.profile["DS_t"]/ 1e13,
    cluster.profile["DS_t_err"]/1e13,
    label='sky'
)
plt.loglog()

ax.set_xlabel("r [Mpc]", fontsize=20)
ax.set_ylabel(r"$\Delta\Sigma(r)$", fontsize=20)
ax.grid(lw=0.3)
ax.minorticks_on()
ax.grid(which="minor", lw=0.1)
plt.legend(fontsize=20)
plt.show()
plt.savefig('clmm_hsc_cluster.png')


Measuring the lensing signal without using CLMM.

In [ ]:
import numpy as np
from astropy.io import fits
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import spatial
from colossus.cosmology import cosmology

In [ ]:
clust = np.genfromtxt('./data_coords/mini_GAMA15H/redmapper_dr8_GAMA15H.txt',dtype=np.dtype([('RA',np.float64),('Dec',np.float64),('z',np.float64),('richness',np.float64)]))
source = fits.getdata('./data_coords/mini_GAMA15H/GAMA15H_tutorial.fits')

In [ ]:
cosmo = cosmology.setCosmology('planck18')

In [ ]:
#Define a function to compute the angle between lens and source galaxies
def calcDistanceAngle(a1, d1, a2, d2):
    return np.arccos(np.cos(d1)*np.cos(d2)*np.cos(a1-a2)+np.sin(d1)*np.sin(d2))

#Define a function to compute the cos(2phi) and sin(2phi) between lens and source galaxies
def cosPhi2(a1, a2, d1, d2, distanceAngle):
    return np.sin(a2 - a1)*np.cos(d1)/np.sin(distanceAngle)

def sinPhi2(a1, a2, d1, d2, distanceAngle):
    return (-np.cos(d2)*np.sin(d1) + np.sin(d2)*np.cos(d1)*np.cos(a1-a2))/np.sin(distanceAngle)

In [ ]:
#TS: This is a simplified way to measure lensing signals (without worrying about resposivity, multiplicative and additive biases)
d2r = np.pi/180.
r2d = 180./np.pi
Mpc = 3.08568025*10**19 # 1Mpc = 3.08568025*10**19 km                                                                                                    
M_sun = 1.9884*10**33 # solar mass [g]  
c_light = 2.99792458*10**5 # speed of light [km/s] 
G = 6.67384*10**(-20)# gravitational constant [km^3s^-2kg^-1] 
Sigma_cr_fact = c_light**2/(4*np.pi*G)*Mpc*10**3/M_sun
bin_edges = np.logspace(-1,np.log10(20),15)#Define your radial bins

sum = np.rec.fromarrays((np.zeros(len(bin_edges)-1),
                         np.zeros(len(bin_edges)-1),
                         np.zeros(len(bin_edges)-1),
                         np.zeros(len(bin_edges)-1),
                         np.zeros(len(bin_edges)-1),
                         np.zeros(len(bin_edges)-1),
                         np.zeros(len(bin_edges)-1),
                         np.zeros(len(bin_edges)-1),
                         np.zeros(len(bin_edges)-1)),
                         names = ("e_sq", "w", "wetsigma", "wetsigma_sq", "wexsigma", "wexsigma_sq", "wsigmainv", "wzs", "n"))


#Calculate tangential shear for each lens
#Select lens clusters: selecting many will take time to compute...
source1 = source[source['photoz']>0.5]
tree = spatial.cKDTree(np.array((source1['RA'],source1['Dec'])).T)


source2 = source[(source['photoz']<1.0)&(source['photoz']>0.9)]
tree = spatial.cKDTree(np.array((source1['RA'],source1['Dec'])).T)
l = clust[(clust['z']>0.1)&(clust['z']<0.2)][:1]  
print(l.size)
l_weight = np.ones(l.size) #Weights for the cluster lenses are one

for i in np.arange(l.size):

    #Select source galaxies
    sel = tree.query_ball_point([l['RA'][i],l['Dec'][i]], 3)
    sel_z = source[sel]['photoz']>l['z'][i] #Try to change the source galaxy selection
    source_bg = source[sel][sel_z]
    theta = calcDistanceAngle(l['RA'][i]*d2r, l['Dec'][i]*d2r, source_bg['RA']*d2r, source_bg['Dec']*d2r)  #Compute an angle between the lens and the source
    l_chi =  cosmo.comovingDistance(z_min=0.0,z_max=l['z'][i])#Compute the comoving distance of the lens
    r = theta*l_chi                                                                                                 
    assign_r = np.digitize(r, bin_edges)


    for i_r in range(len(bin_edges)-1):
        #print i_r                                                                                                                               
        sel = assign_r == i_r+1
        source_bg_inbin = source_bg[sel]
        theta_sub = theta[sel]
        cosp = cosPhi2(l["RA"][i]*d2r, source_bg_inbin["RA"]*d2r, l["Dec"][i]*d2r, source_bg_inbin["Dec"]*d2r, theta_sub)
        sinp = sinPhi2(l["RA"][i]*d2r, source_bg_inbin["RA"]*d2r, l["Dec"][i]*d2r, source_bg_inbin["Dec"]*d2r, theta_sub)
        cos2p = cosp**2 - sinp**2
        sin2p = 2.*sinp*cosp
        et = -(source_bg_inbin["e1"]*cos2p+source_bg_inbin["e2"]*sin2p)
        ex = -(-source_bg_inbin["e1"]*sin2p+source_bg_inbin["e2"]*cos2p)
        # critical surface mass density [M_sun/ comoving Mpc^2]. (1+zl)**-2 is for comoving coordinates.  
        Sigma_cr = Sigma_cr_fact/(1. - l_chi/cosmo.comovingDistance(z_min=0.0,z_max=source_bg_inbin["photoz"]))/l_chi/(1.+l["z"][i])/10**12  
                              
        sum["n"][i_r] += sel.sum()
                              
        sum["e_sq"][i_r] += np.sum(et**2 + ex**2)
        wt = source_bg_inbin["weight"]*Sigma_cr**-2 #multiply by the lens weights if it is not one
        sum["w"][i_r] += np.sum(wt)
        wetsigma = wt*Sigma_cr*et
        sum["wetsigma"][i_r] += np.sum(wetsigma)
        sum["wetsigma_sq"][i_r] += np.sum(wetsigma**2)
        wexsigma = wt*Sigma_cr*ex
        sum["wexsigma"][i_r] += np.sum(wexsigma)
        sum["wexsigma_sq"][i_r] += np.sum(wexsigma**2)
        wsigmainv = wt*1./Sigma_cr
        sum["wsigmainv"][i_r] += np.sum(wsigmainv)
        wzs = wt*source_bg_inbin["photoz"]
        sum["wzs"][i_r] += np.sum(wzs)

In [ ]:
#Compute the lensing signal            
radial_bin = 2.*(bin_edges[1:]**3 - bin_edges[:-1]**3)/(3.*(bin_edges[1:]**2 - bin_edges[:-1]**2))
gt = 0.5*sum["wetsigma"]/sum["w"]
gt_err = 0.5*np.sqrt(sum["wetsigma_sq"])/sum["w"]
gx = 0.5*sum["wexsigma"]/sum["w"]
gx_err = 0.5*np.sqrt(sum["wexsigma_sq"])/sum["w"]
sigma_cr = 1./(sum["wsigmainv"]/sum["w"])

In [ ]:
#Figure for the lensing signal
plt.figure(figsize=(6,6))
plt.errorbar(radial_bin,gt,yerr=gt_err,label='original')
plt.errorbar(
    cluster.profile["radius"],
    cluster.profile["DS_t"]/ 1e13,
    cluster.profile["DS_t_err"]/1e13,
    label='CLMM'
)
plt.loglog()
plt.legend(fontsize=20)
plt.xlabel(r'$R[h^{-1}{\rm Mpc}]$',fontsize=20)
plt.ylabel(r'$\Delta\Sigma(R)$',fontsize=20)
plt.savefig('weaklens_hsc_clust.png')